# Lab 1 

Laboration 1 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-1/

Authour: Jonatan Kronander - elt15jkr@student.lu.se

### The objectives of this assignment are to:

-Write a program that collects all the words from a set of documents

-Build an index from the words

-Know what indexing is

-Represent a document using the Tf.Idf value

-Write a short report of 1 to 2 pages on the assignment

-Read a short text on an industrial system

## Indexing one file

* The index file will contain all the unique words in the document, where each word is associated with the list of its positions in the document.

* You will represent this index as a dictionary where the keys will be the words and the values, the lists of positions

* As words, you will consider all the strings of letters that you will set in lower case. You will not index the rest (i.e. numbers or symbols).

This is done by first using function txtClean.

In [3]:
import re

def txtClean(text):
    """
    Replace capital characters to small characters
    
    Input txt file
    Output txt file
    """
    # Remove new lines
    text = re.sub("\n", " ", text)
    
    # Replace [A-Ö] with [a-ö]
    def toLowercase(matchobj):
        return matchobj.group(1).lower()
    
    text = re.sub(r'([A-Z])', toLowercase, text)
    
    # Remove multiple spaces
    text = re.sub(' +', ' ', text)

    return text    

In [4]:
text = open("Selma/bannlyst.txt").read()  
txt = txtClean(text)

* To extract the words, you will use Unicode regular expressions. Do not use \w+, for instance, but the Unicode equivalent. The word positions will correspond to the number of characters from the beginning of the file. (The word offset from the beginning)

* You will use finditer() to find the positions of the words. This will return you match objects, where you will get the matches and the positions with the group() and start() methods.

In [5]:
def string2dict(text,originaltext):
    """
    Creates a dict with (word:list[index apperences]) from input string
    
    Input string, string
    Output dict
    """
    stringList = re.findall(r"[a-zåäö]+",text) #This finds all words from a txt file. r"[a-zåäö]+ equal to r"\w+" 

    stringDict = dict.fromkeys(stringList) #Creates dict (and remove doublicates)

    for word in stringDict:
        wordIndices = []
        pattern = r"\b"+word+ r"\b" #Only look at word 
        
        for m in re.finditer(pattern, originaltext): #Iterate thorugh every word
            wordIndices.append(m.start(0))
            
        stringDict.update({word:wordIndices})
    return stringDict

In [6]:
txtDict = string2dict(txt,text) 

In [7]:
len(txtDict)

7950

#### Test with bannlyst text
The word gjord occurs three times in the text at positions 8551, 183692, and 220875, uppklarnande, once at position 8567, and stjärnor, once at position 8590. 

In [8]:
txtDict['gjord']

[8551, 183692, 220875]

In [9]:
txtDict['uppklarnande']

[8567]

In [10]:
txtDict['stjärnor']

[8590]

#### Pickle

* You will use the pickle package to write your dictionary in an file, see https://wiki.python.org/moin/UsingPickle.

In [11]:
import pickle

with open('BannlystTxtDict.pickle', 'wb') as handle:
    pickle.dump(txtDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Open pickle

In [12]:
with open('BannlystTxtDict.pickle', 'rb') as handle:
    BannlystTxtDict = pickle.load(handle)

#### Test of pickle

In [13]:
BannlystTxtDict == txtDict

True

## Reading the content of a folder

Write a function that reads all the files in a folder with a specific suffix (txt). You will need the Python os package, see https://docs.python.org/3/library/os.html. You will return the file names in a list.

Use function:

In [17]:
import os

def get_files(fileDir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param filedir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(fileDir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [18]:
files = get_files("selma", ".txt")

In [19]:
files

['troll.txt',
 'kejsaren.txt',
 'marbacka.txt',
 'herrgard.txt',
 'nils.txt',
 'osynliga.txt',
 'jerusalem.txt',
 'bannlyst.txt',
 'gosta.txt']

## Creating a master index

Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions. (a posting list)

In [157]:
def toLowercase(matchobj):
    return matchobj.group(1).lower()
    
def addAll(fileDir,files): 
    """
    Reads all files in list and matches to txt files
    :param dir:
    :param files:
    :return dict:
    """
    allDict = {}

    for file in files:
        text = open(fileDir+"/"+file).read()  
        txt = txtClean(text)
        stringList = re.findall(r"[a-zåäö]+",txt)
        allDict.update(dict.fromkeys(stringList)) # is this ok?!
    
    wordIndices = []  
    i = 0

    for word in allDict: # Iterate through every word in master dict  
        pattern = r"\b"+word+ r"\b" # Only look at word
        allDict[word] = {}
        
        for file in files: # Iterate through every text
            text = re.sub(r'([A-Z])', toLowercase, open(fileDir+"/"+file).read()) # Open text and lowercase all
            
            for m in re.finditer(pattern, text): # Iterate through every word in file text
                wordIndices.append(m.start(0))  
                
            allDict[word][file] = list(wordIndices)
            wordIndices.clear()
        print(i/len(allDict))
        i = i+1
      
    return allDict

In [158]:
masterDict = addAll('selma',files)

0.0
2.53350561171493e-05
5.06701122342986e-05
7.60051683514479e-05
0.0001013402244685972
0.0001266752805857465
0.0001520103367028958
0.0001773453928200451
0.0002026804489371944
0.0002280155050543437
0.000253350561171493
0.00027868561728864227
0.0003040206734057916
0.0003293557295229409
0.0003546907856400902
0.00038002584175723947
0.0004053608978743888
0.0004306959539915381
0.0004560310101086874
0.0004813660662258367
0.000506701122342986
0.0005320361784601353
0.0005573712345772845
0.0005827062906944339
0.0006080413468115832
0.0006333764029287325
0.0006587114590458818
0.000684046515163031
0.0007093815712801804
0.0007347166273973297
0.0007600516835144789
0.0007853867396316283
0.0008107217957487776
0.0008360568518659269
0.0008613919079830762
0.0008867269641002254
0.0009120620202173748
0.0009373970763345241
0.0009627321324516734
0.0009880671885688227
0.001013402244685972
0.0010387373008031212
0.0010640723569202706
0.00108940741303742
0.001114742469154569
0.0011400775252717185
0.001165412581

0.009804666717336779
0.009830001773453929
0.009855336829571078
0.009880671885688226
0.009906006941805377
0.009931341997922526
0.009956677054039674
0.009982012110156825
0.010007347166273974
0.010032682222391122
0.010058017278508273
0.010083352334625421
0.01010868739074257
0.01013402244685972
0.01015935750297687
0.010184692559094018
0.010210027615211168
0.010235362671328317
0.010260697727445466
0.010286032783562616
0.010311367839679765
0.010336702895796914
0.010362037951914064
0.010387373008031213
0.010412708064148362
0.010438043120265512
0.01046337817638266
0.01048871323249981
0.01051404828861696
0.010539383344734109
0.010564718400851257
0.010590053456968408
0.010615388513085557
0.010640723569202705
0.010666058625319856
0.010691393681437004
0.010716728737554153
0.010742063793671304
0.010767398849788452
0.010792733905905601
0.010818068962022752
0.0108434040181399
0.010868739074257049
0.0108940741303742
0.010919409186491348
0.010944744242608497
0.010970079298725647
0.010995414354842796
0.

0.0198373489397279
0.01986268399584505
0.019888019051962202
0.01991335410807935
0.0199386891641965
0.01996402422031365
0.019989359276430797
0.020014694332547947
0.020040029388665098
0.020065364444782244
0.020090699500899395
0.020116034557016545
0.020141369613133692
0.020166704669250843
0.020192039725367993
0.02021737478148514
0.02024270983760229
0.02026804489371944
0.020293379949836588
0.02031871500595374
0.02034405006207089
0.020369385118188036
0.020394720174305186
0.020420055230422337
0.020445390286539484
0.020470725342656634
0.020496060398773785
0.020521395454890932
0.020546730511008082
0.020572065567125233
0.02059740062324238
0.02062273567935953
0.02064807073547668
0.020673405791593828
0.020698740847710978
0.02072407590382813
0.020749410959945275
0.020774746016062426
0.020800081072179576
0.020825416128296723
0.020850751184413874
0.020876086240531024
0.02090142129664817
0.02092675635276532
0.020952091408882472
0.02097742646499962
0.02100276152111677
0.02102809657723392
0.02105343163

0.029920701274353322
0.029946036330470473
0.02997137138658762
0.02999670644270477
0.03002204149882192
0.030047376554939068
0.030072711611056218
0.03009804666717337
0.030123381723290515
0.030148716779407666
0.030174051835524816
0.030199386891641963
0.030224721947759114
0.030250057003876264
0.03027539205999341
0.03030072711611056
0.030326062172227712
0.03035139722834486
0.03037673228446201
0.03040206734057916
0.030427402396696307
0.030452737452813457
0.030478072508930608
0.030503407565047755
0.030528742621164905
0.030554077677282056
0.030579412733399206
0.030604747789516353
0.030630082845633504
0.030655417901750654
0.0306807529578678
0.03070608801398495
0.030731423070102102
0.03075675812621925
0.0307820931823364
0.03080742823845355
0.030832763294570697
0.030858098350687847
0.030883433406804998
0.030908768462922145
0.030934103519039295
0.030959438575156446
0.030984773631273593
0.031010108687390743
0.031035443743507894
0.03106077879962504
0.03108611385574219
0.03111144891185934
0.031136783

0.04018139900179879
0.04020673405791594
0.04023206911403309
0.04025740417015024
0.040282739226267385
0.04030807428238454
0.040333409338501686
0.04035874439461883
0.04038407945073599
0.040409414506853134
0.04043474956297028
0.040460084619087434
0.04048541967520458
0.04051075473132173
0.04053608978743888
0.04056142484355603
0.040586759899673176
0.04061209495579033
0.04063743001190748
0.040662765068024624
0.04068810012414178
0.040713435180258925
0.04073877023637607
0.040764105292493226
0.04078944034861037
0.04081477540472752
0.040840110460844674
0.04086544551696182
0.04089078057307897
0.04091611562919612
0.04094145068531327
0.040966785741430416
0.04099212079754757
0.04101745585366472
0.041042790909781864
0.04106812596589902
0.041093461022016164
0.04111879607813331
0.041144131134250465
0.04116946619036761
0.04119480124648476
0.04122013630260191
0.04124547135871906
0.04127080641483621
0.04129614147095336
0.04132147652707051
0.041346811583187655
0.04137214663930481
0.041397481695421956
0.041

...

0.9480124648476096
0.9480377999037268
0.948063134959844
0.9480884700159611
0.9481138050720782
0.9481391401281953
0.9481644751843126
0.9481898102404297
0.9482151452965468
0.948240480352664
0.9482658154087811
0.9482911504648983
0.9483164855210154
0.9483418205771326
0.9483671556332497
0.9483924906893669
0.948417825745484
0.9484431608016012
0.9484684958577183
0.9484938309138354
0.9485191659699527
0.9485445010260698
0.9485698360821869
0.948595171138304
0.9486205061944212
0.9486458412505384
0.9486711763066555
0.9486965113627727
0.9487218464188898
0.9487471814750069
0.9487725165311242
0.9487978515872413
0.9488231866433584
0.9488485216994755
0.9488738567555927
0.9488991918117099
0.948924526867827
0.9489498619239441
0.9489751969800613
0.9490005320361785
0.9490258670922956
0.9490512021484128
0.9490765372045299
0.949101872260647
0.9491272073167643
0.9491525423728814
0.9491778774289985
0.9492032124851156
0.9492285475412328
0.94925388259735
0.9492792176534671
0.9493045527095842
0.9493298877657014
0

0.9590078792024525
0.9590332142585696
0.9590585493146867
0.9590838843708038
0.9591092194269211
0.9591345544830382
0.9591598895391553
0.9591852245952724
0.9592105596513897
0.9592358947075068
0.9592612297636239
0.9592865648197411
0.9593118998758582
0.9593372349319754
0.9593625699880926
0.9593879050442097
0.9594132401003268
0.9594385751564439
0.9594639102125612
0.9594892452686783
0.9595145803247954
0.9595399153809125
0.9595652504370297
0.9595905854931469
0.959615920549264
0.9596412556053812
0.9596665906614983
0.9596919257176154
0.9597172607737326
0.9597425958298498
0.9597679308859669
0.959793265942084
0.9598186009982013
0.9598439360543184
0.9598692711104355
0.9598946061665526
0.9599199412226698
0.959945276278787
0.9599706113349041
0.9599959463910213
0.9600212814471384
0.9600466165032555
0.9600719515593727
0.9600972866154899
0.960122621671607
0.9601479567277241
0.9601732917838413
0.9601986268399585
0.9602239618960756
0.9602492969521927
0.9602746320083099
0.960299967064427
0.960325302120544

0.9700032935572952
0.9700286286134123
0.9700539636695296
0.9700792987256467
0.9701046337817638
0.9701299688378809
0.9701553038939982
0.9701806389501153
0.9702059740062324
0.9702313090623496
0.9702566441184667
0.9702819791745839
0.970307314230701
0.9703326492868182
0.9703579843429353
0.9703833193990524
0.9704086544551697
0.9704339895112868
0.9704593245674039
0.970484659623521
0.9705099946796382
0.9705353297357554
0.9705606647918725
0.9705859998479897
0.9706113349041068
0.9706366699602239
0.9706620050163411
0.9706873400724583
0.9707126751285754
0.9707380101846925
0.9707633452408098
0.9707886802969269
0.970814015353044
0.9708393504091611
0.9708646854652783
0.9708900205213955
0.9709153555775126
0.9709406906336298
0.9709660256897469
0.970991360745864
0.9710166958019812
0.9710420308580984
0.9710673659142155
0.9710927009703326
0.9711180360264497
0.971143371082567
0.9711687061386841
0.9711940411948012
0.9712193762509184
0.9712447113070355
0.9712700463631527
0.9712953814192699
0.971320716475387

0.9810240429682552
0.9810493780243723
0.9810747130804894
0.9811000481366067
0.9811253831927238
0.9811507182488409
0.981176053304958
0.9812013883610752
0.9812267234171924
0.9812520584733095
0.9812773935294267
0.9813027285855438
0.9813280636416609
0.9813533986977782
0.9813787337538953
0.9814040688100124
0.9814294038661295
0.9814547389222467
0.9814800739783639
0.981505409034481
0.9815307440905982
0.9815560791467153
0.9815814142028325
0.9816067492589496
0.9816320843150668
0.9816574193711839
0.981682754427301
0.9817080894834183
0.9817334245395354
0.9817587595956525
0.9817840946517696
0.9818094297078868
0.981834764764004
0.9818600998201211
0.9818854348762383
0.9819107699323554
0.9819361049884725
0.9819614400445897
0.9819867751007069
0.982012110156824
0.9820374452129411
0.9820627802690582
0.9820881153251755
0.9821134503812926
0.9821387854374097
0.9821641204935269
0.9821894555496441
0.9822147906057612
0.9822401256618783
0.9822654607179955
0.9822907957741126
0.9823161308302298
0.982341465886347

0.9920194573230979
0.9920447923792152
0.9920701274353323
0.9920954624914494
0.9921207975475665
0.9921461326036837
0.9921714676598009
0.992196802715918
0.9922221377720352
0.9922474728281523
0.9922728078842694
0.9922981429403867
0.9923234779965038
0.9923488130526209
0.992374148108738
0.9923994831648552
0.9924248182209724
0.9924501532770895
0.9924754883332066
0.9925008233893238
0.992526158445441
0.9925514935015581
0.9925768285576753
0.9926021636137924
0.9926274986699095
0.9926528337260268
0.9926781687821439
0.992703503838261
0.9927288388943781
0.9927541739504953
0.9927795090066125
0.9928048440627296
0.9928301791188467
0.9928555141749639
0.992880849231081
0.9929061842871982
0.9929315193433154
0.9929568543994325
0.9929821894555496
0.9930075245116667
0.993032859567784
0.9930581946239011
0.9930835296800182
0.9931088647361354
0.9931341997922526
0.9931595348483697
0.9931848699044868
0.993210204960604
0.9932355400167211
0.9932608750728383
0.9932862101289555
0.9933115451850726
0.9933368802411897


In [160]:
#with open('masterDict1.pickle', 'wb') as handle:
#    pickle.dump(masterDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [206]:
with open('masterDict1.pickle', 'rb') as handle:
    masterDict = pickle.load(handle)

#### Test of master dict. Below is an except of the master index with the words samlar and ände:

'samlar': {'nils.txt': [53499, 120336], 'gosta.txt': [317119, 414300, 543686],'osynliga.txt': [410995, 871322]},

'ände':{'nils.txt': [3991],'kejsaren.txt': [51100],'marbacka.txt': [374231],'troll.txt': [39726],'osynliga.txt': [742747]},

In [161]:
masterDict["samlar"]

{'troll.txt': [],
 'kejsaren.txt': [],
 'marbacka.txt': [],
 'herrgard.txt': [],
 'nils.txt': [53499, 120336],
 'osynliga.txt': [410995, 871322],
 'jerusalem.txt': [],
 'bannlyst.txt': [],
 'gosta.txt': [317119, 414300, 543686]}

In [162]:
masterDict["ände"]

{'troll.txt': [39726],
 'kejsaren.txt': [51100],
 'marbacka.txt': [374231],
 'herrgard.txt': [],
 'nils.txt': [3991],
 'osynliga.txt': [],
 'jerusalem.txt': [],
 'bannlyst.txt': [],
 'gosta.txt': []}

In [163]:
masterDict['nils']

{'troll.txt': [276444],
 'kejsaren.txt': [265284, 268547],
 'marbacka.txt': [351092, 369109],
 'herrgard.txt': [],
 'nils.txt': [0,
  19889,
  19926,
  51386,
  62890,
  100616,
  101011,
  102195,
  102412,
  108926,
  123890,
  140291,
  160578,
  188696,
  205783,
  206296,
  216382,
  281361,
  285415,
  285512,
  292520,
  292749,
  293016,
  294829,
  303563,
  308289,
  317408,
  333414,
  360698,
  405161,
  412732,
  458051,
  501193,
  560856,
  578545,
  599267,
  623076,
  633791,
  664519,
  664779,
  671361,
  679424,
  692312,
  698533,
  698890,
  700773,
  704606,
  756218,
  775386,
  861869,
  861971,
  863312,
  864324,
  869574,
  932839,
  986808,
  987178,
  1005814,
  1013735,
  1014067,
  1014478,
  1039517,
  1057645,
  1058416,
  1059387,
  1062721,
  1072927,
  1073081,
  1073558,
  1075687,
  1076491,
  1076639,
  1078793,
  1079876,
  1080069,
  1084585],
 'osynliga.txt': [],
 'jerusalem.txt': [173228, 327678, 327726],
 'bannlyst.txt': [],
 'gosta.txt': []

## Representing Documents with tf-idf

Once you have created the index, you will represent each document in your corpus as a word vector. You will define the value of a word in a document with the tf-idf metric. Tf will be the relative frequency of the term in the document and idf, the logarithm base 10 of the inverse document frequency.

In [ ]:
import math

def tiIdf(masterDict):
    tfIdfDict = masterDict.copy()
    
    for word in masterDict: 
        #idf will be the logarithm base 10 of the inverse document frequency.
        nbrKeys = len(masterDict[word].keys())
        dictValues = masterDict[word].values()
        lenDictValues = len(dictValues)
        
        i = 0
        for fileList in dictValues:
            if not fileList:
                i = i + 1

        df = (lenDictValues-i)
        if df == 0:
            idf = 0
        else:
            idf = math.log10(nbrKeys/df)
        
        for file in masterDict[word]:
            # Tf will be the relative frequency of the term in the document 
            text = open('selma'+"/"+file).read()  
            txt = txtClean(text)
            stringList = re.findall(r"[a-zåäö]+",text)
            lenText = len(stringList) # nbr of words in textfile
            
            lenWordVec = len(masterDict[word][file]) # nbr of occurencies of word

            tf =  lenWordVec / lenText   
            tfIdfDict[word][file] = tf*idf
    return tfIdfDict
        

In [ ]:
tfIdfDict = tiIdf(masterDict)

#### Test of tf idf:

känna :: bannlyst.txt 0.0, gosta.txt 0.0, herrgard.txt 0.0, jerusalem.txt 0.0, nils.txt 0.0

In [194]:
tfIdfDict['känna']

{'troll.txt': [71489,
  125715,
  152491,
  190662,
  198550,
  199126,
  219519,
  223883,
  348650,
  367362],
 'kejsaren.txt': [4516, 52476, 109515, 192974, 196116, 214277],
 'marbacka.txt': [129831,
  130153,
  135178,
  162444,
  263151,
  317247,
  345052,
  373500,
  377026,
  379828],
 'herrgard.txt': [9657,
  10076,
  11774,
  20331,
  41763,
  48573,
  60784,
  77783,
  106744,
  107548,
  131534,
  179279,
  182804],
 'nils.txt': [24481,
  117169,
  133648,
  185171,
  186391,
  186550,
  187124,
  193832,
  271440,
  324927,
  350478,
  393097,
  481442,
  523515,
  528965,
  539843,
  558775,
  558998,
  623679,
  693933,
  709020,
  752403,
  768003,
  780605,
  818708,
  835567,
  839819,
  852549,
  957647,
  959404,
  995408,
  996108,
  1018540,
  1018871,
  1025793,
  1036220,
  1042694,
  1055914,
  1063048,
  1064609,
  1081247,
  1088489],
 'osynliga.txt': [43390,
  50556,
  51968,
  59063,
  59220,
  59814,
  129517,
  148972,
  155124,
  155968,
  156269,
  3037

gås :: bannlyst.txt 0.0, gosta.txt 0.0, herrgard.txt 0.0, jerusalem.txt 0.0, nils.txt 0.00010123719421964931

In [169]:
tfIdfDict['gås']

{'troll.txt': [],
 'kejsaren.txt': [],
 'marbacka.txt': [],
 'herrgard.txt': [],
 'nils.txt': [55009,
  55420,
  57096,
  63099,
  65078,
  89801,
  117700,
  147664,
  148039,
  218658,
  227740,
  258358,
  259678,
  404562,
  404702,
  404790,
  532929,
  660797,
  662609,
  721397,
  1014194],
 'osynliga.txt': [],
 'jerusalem.txt': [],
 'bannlyst.txt': [],
 'gosta.txt': []}

nils :: bannlyst.txt 0.0, gosta.txt 0.0, herrgard.txt 0.0 jerusalem.txt 4.778415355159037e-06, nils.txt 9.801209641132888e-05

In [209]:
tfIdfDict['nils']

{'troll.txt': 3.6624988178210027e-06,
 'kejsaren.txt': 8.107749884176785e-06,
 'marbacka.txt': 7.5972829304119305e-06,
 'herrgard.txt': 0.0,
 'nils.txt': 9.81613652422891e-05,
 'osynliga.txt': 0.0,
 'jerusalem.txt': 4.784864200624293e-06,
 'bannlyst.txt': 0.0,
 'gosta.txt': 0.0}

et bannlyst.txt 6.2846093167673765e-06, gosta.txt 0.0, herrgard.txt 0.0, jerusalem.txt 0.0, nils.txt 0.0

In [ ]:
masterDict['et']

## Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tfidf representation and present your results in a matrix. You will include this matrix in your report.

Give the name of the two novels that are the most similar.

In [ ]:
from scipy import spatial

result = 1 - spatial.distance.cosine(dataSetI, dataSetII)



## Reading

Read the text: Challenges in Building Large-Scale Information Retrieval Systems about the history of Google indexing by Jeff Dean. In your report, tell how your index encoding is related to what Google did. You must identify the slide where you have the most similar indexing technique and write the slide number in your report. https://static.googleusercontent.com/media/research.google.com/en//people/jeff/WSDM09-keynote.pdf

#### Answer: